In [2]:
import pickle
import os
import pickle
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from pandas.io import gbq
from google.cloud import bigquery
from google.oauth2 import service_account

%matplotlib inline
pd.set_option('display.max_colwidth', -1)
% config InlineBackend . figure_format = 'svg'

In [3]:
# This cell is just about making things look nice
# Make plots bigger by default.
# Has to occur after the %matplotlib inline in a different cell. Known issue
plt.rc('figure', figsize=(12.0, 6.0))
matplotlib.rcParams.update({'font.size': 22})

# Make numpy outputs easier to read
np.set_printoptions(precision=3, formatter={'float': '{: 8.3f}'.format})

In [4]:
## Set up bigquery environment

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/ubuntu/certs/stackoverflow-metis-32ecb3fa04c0.json"
project_id = 'stackoverflow-metis'
client = bigquery.Client(project=project_id)
dataset_id = 'SO_dataset'
dataset_ref = client.dataset(dataset_id)
dataset = client.get_dataset(dataset_ref)

## Create a view to reconstruct user reputation at 01/01/2017 and 01/01/2018


In [ ]:
## Create a view to reconstruct user reputation at 01/01/2017 and 01/01/2018
view_ref = dataset_ref.table('SO_user_reputation_view')
view = bigquery.Table(view_ref)

view.view_query = """
#standardSQL
SELECT
  user.id,
  LENGTH(about_me) AS about_me_length,
  user.age,
  user.creation_date,
  user.last_access_date,
  user.location,
  user.reputation,
  user.up_votes,
  user.down_votes,
  user.profile_image_url,
  user.website_url,
  answer_2017.answer_reputation_total_2017,
  question_2017.question_reputation_total_2017,
  accepted_2017.accepted_reputation_total_2017,
  answer_2018.answer_reputation_total_2018,
  question_2018.question_reputation_total_2018,
  accepted_2018.accepted_reputation_total_2018
FROM
  `bigquery-public-data.stackoverflow.users` AS user
LEFT JOIN (
  SELECT
    owner_user_id,
    SUM(answer_reputation) AS answer_reputation_total_2017
  FROM (
    SELECT
      owner_user_id,
      score * 10 AS answer_reputation
    FROM
      `bigquery-public-data.stackoverflow.posts_answers`
    WHERE
      creation_date < '2017-01-01 00:00:00')
  GROUP BY
    owner_user_id) AS answer_2017
ON
  user.id = answer_2017.owner_user_id
LEFT JOIN (
  SELECT
    owner_user_id,
    SUM(question_reputation) AS question_reputation_total_2017
  FROM (
    SELECT
      owner_user_id,
      score * 5 AS question_reputation
    FROM
      `bigquery-public-data.stackoverflow.posts_questions`
    WHERE
      creation_date < '2017-01-01 00:00:00')
  GROUP BY
    owner_user_id) AS question_2017
ON
  user.id = question_2017.owner_user_id
LEFT JOIN (
  SELECT
    owner_user_id,
    SUM(accepted_reputation) AS accepted_reputation_total_2017
  FROM (
    SELECT
      answer.owner_user_id,
      CASE
        WHEN answer.id = question.accepted_answer_id THEN 15
        ELSE 0
      END AS accepted_reputation
    FROM
      `bigquery-public-data.stackoverflow.posts_answers` AS answer
    LEFT JOIN
      `bigquery-public-data.stackoverflow.posts_questions` question
    ON
      answer.parent_id = question.id
    WHERE
      answer.creation_date < '2017-01-01 00:00:00')
  GROUP BY
    owner_user_id) AS accepted_2017
ON
  user.id = accepted_2017.owner_user_id
LEFT JOIN (
  SELECT
    owner_user_id,
    SUM(answer_reputation) AS answer_reputation_total_2018
  FROM (
    SELECT
      owner_user_id,
      score * 10 AS answer_reputation
    FROM
      `bigquery-public-data.stackoverflow.posts_answers`
    WHERE
      creation_date < '2018-01-01 00:00:00')
  GROUP BY
    owner_user_id) AS answer_2018
ON
  user.id = answer_2018.owner_user_id
LEFT JOIN (
  SELECT
    owner_user_id,
    SUM(question_reputation) AS question_reputation_total_2018
  FROM (
    SELECT
      owner_user_id,
      score * 5 AS question_reputation
    FROM
      `bigquery-public-data.stackoverflow.posts_questions`
    WHERE
      creation_date < '2018-01-01 00:00:00')
  GROUP BY
    owner_user_id) AS question_2018
ON
  user.id = question_2018.owner_user_id
LEFT JOIN (
  SELECT
    owner_user_id,
    SUM(accepted_reputation) AS accepted_reputation_total_2018
  FROM (
    SELECT
      answer.owner_user_id,
      CASE
        WHEN answer.id = question.accepted_answer_id THEN 15
        ELSE 0
      END AS accepted_reputation
    FROM
      `bigquery-public-data.stackoverflow.posts_answers` AS answer
    LEFT JOIN
      `bigquery-public-data.stackoverflow.posts_questions` question
    ON
      answer.parent_id = question.id
    WHERE
      answer.creation_date < '2018-01-01 00:00:00')
  GROUP BY
    owner_user_id) AS accepted_2018
ON
  user.id = accepted_2018.owner_user_id
"""
view = client.create_table(view)

In [13]:
## Retrieve data from the view
query_job = client.query("""
    SELECT *
    FROM `stackoverflow-metis.SO_dataset.SO_user_reputation_view`
    """)

results = query_job.result()
user_reputation = results.to_dataframe()

In [23]:
## Process and save the reputation data
user_reputation['reputation_2017'] = user_reputation[['answer_reputation_total_2017','question_reputation_total_2017','accepted_reputation_total_2017']].sum(axis=1)
user_reputation['reputation_2018'] = user_reputation[['answer_reputation_total_2018','question_reputation_total_2018','accepted_reputation_total_2018']].sum(axis=1)
with open('./data/processed/user_reputation.pkl', 'wb') as picklefile:
    pickle.dump(user_reputation, picklefile)

In [4]:
with open('./data/processed/user_reputation.pkl', 'rb') as picklefile:
    user_reputation = pickle.load(picklefile)

In [5]:
user_reputation.tail(100)

id  about_me_length age                    creation_date  \
8917407  1525495  98                  2012-07-14 11:28:27.330000+00:00   
8917408  1811348  0                   2012-11-09 05:53:59.980000+00:00   
8917409  379532   0                   2010-06-29 22:54:24.830000+00:00   
8917410  2985898  0                   2013-11-13 03:01:55.323000+00:00   
8917411  1816631  0                   2012-11-11 20:21:58.540000+00:00   
8917412  801169   0                   2011-06-16 09:34:42.970000+00:00   
8917413  22227    0                   2008-09-25 15:40:21.703000+00:00   
8917414  105668   0                   2009-05-12 20:05:21.497000+00:00   
8917415  1062015  0                   2011-11-23 13:41:45.487000+00:00   
8917416  187523   0                   2009-10-10 02:37:21.080000+00:00   
8917417  1619521  0                   2012-08-23 10:56:58.513000+00:00   
8917418  1105571  0                   2011-12-19 09:29:25.343000+00:00   
8917419  721756   0                   2011-04-23 13:20:50.633000+00:00   
8917420  1158470  0                   2012-01-19 12:31:43.723000+00:00   
8917421  1185737  133                 2012-02-02 17:17:42.773000+00:00   
8917422  3829     0                   2008-08-31 04:59:21+00:00          
8917423  1816253  0                   2012-11-11 15:42:31.123000+00:00   
8917424  301744   464                 2010-03-25 14:00:33.173000+00:00   
8917425  450117   613                 2010-09-16 23:52:20.397000+00:00   
8917426  122777   406                 2009-06-14 16:57:59.227000+00:00   
8917427  1839128  0                   2012-11-20 14:40:17.150000+00:00   
8917428  717336   0                   2011-04-20 14:26:57.013000+00:00   
8917429  1302995  0                   2012-03-30 10:15:29.620000+00:00   
8917430  3652210  204                 2014-05-19 10:24:00.110000+00:00   
8917431  49553    0                   2008-12-28 15:41:09.417000+00:00   
8917432  1137282  0                   2012-01-08 16:25:14.830000+00:00   
8917433  36253    0                   2008-11-10 17:33:17.240000+00:00   
8917434  885453   0                   2011-08-09 07:50:16.037000+00:00   
8917435  599791   0                   2011-02-02 10:05:35.060000+00:00   
8917436  1274685  0                   2012-03-16 18:12:57.180000+00:00   
...          ... ..               ..                               ...   
8917477  4586108  8                   2015-02-19 23:15:31.613000+00:00   
8917478  965338   0                   2011-09-26 15:30:30.913000+00:00   
8917479  109163   0                   2009-05-19 04:52:11.100000+00:00   
8917480  888146   102                 2011-08-10 14:50:08.043000+00:00   
8917481  1366527  182                 2012-04-30 19:08:51.603000+00:00   
8917482  428606   0                   2010-08-23 16:13:39.900000+00:00   
8917483  802815   0                   2011-06-17 07:02:45.137000+00:00   
8917484  646998   129                 2011-03-06 14:46:08.147000+00:00   
8917485  3089468  0                   2013-12-11 03:46:00.150000+00:00   
8917486  1573765  0                   2012-08-03 09:53:03.193000+00:00   
8917487  74298    55                  2009-03-05 15:44:43.627000+00:00   
8917488  3421753  144                 2013-05-06 09:57:19.403000+00:00   
8917489  24065    289                 2008-10-01 09:58:00.233000+00:00   
8917490  1379803  0                   2012-05-07 13:01:58.440000+00:00   
8917491  742819   36                  2011-05-07 07:44:04.077000+00:00   
8917492  82287    0                   2009-03-24 22:55:43.100000+00:00   
8917493  626481   40                  2011-02-21 11:40:42.503000+00:00   
8917494  1279707  0                   2012-03-19 23:05:26.663000+00:00   
8917495  534444   0                   2010-12-08 01:45:34.457000+00:00   
8917496  1491357  0                   2012-06-29 14:05:31.880000+00:00   
8917497  840714   772                 2011-07-12 12:41:13.620000+00:00   
8917498  2114018  0                   2013-02-27 05:38:28.857000+00:00   
8917499  1165637  

## Calculate the time between activities

In [6]:
query = """
#standardSQL
SELECT
  id,
  COUNT(diff) AS count_diff,
  MAX(diff)/24 AS MAX_activity_interval
FROM (
  SELECT
    id,
    TIMESTAMP_DIFF(creation_date, prev_creation_date, HOUR) AS diff
  FROM (
    SELECT
      id,
      creation_date,
      LAG(creation_date) OVER(PARTITION BY id ORDER BY creation_date ) AS prev_creation_date
    FROM (
      SELECT
        *
      FROM (
        SELECT
          owner_user_id AS id,
          creation_date
        FROM
          `bigquery-public-data.stackoverflow.posts_answers`
        UNION ALL
        SELECT
          owner_user_id AS id,
          creation_date
        FROM
          `bigquery-public-data.stackoverflow.posts_questions`
        UNION ALL
        SELECT
          user_id AS id,
          creation_date
        FROM
          `bigquery-public-data.stackoverflow.comments`)
      WHERE
        FORMAT_TIMESTAMP('%Y', creation_date) = '2017')))
GROUP BY
  id
ORDER BY
  COUNT(diff) DESC
"""
query_job = client.query(query)
results = query_job.result()
inter_activity_days = results.to_dataframe()

In [8]:
inter_activity_days.dropna(axis=0, inplace=True)
inter_activity_days.id = inter_activity_days.id.astype(int)
with open('./data/processed/inter_activity_days.pkl', 'wb') as picklefile:
    pickle.dump(inter_activity_days, picklefile)

## 2017 activity count

In [7]:
query = """
#standardSQL
SELECT
id,
count(id) AS count
FROM (
  SELECT
    *
  FROM (
    SELECT
      owner_user_id AS id,
      creation_date
    FROM
      `bigquery-public-data.stackoverflow.posts_answers`
    UNION ALL
    SELECT
      owner_user_id AS id,
      creation_date
    FROM
      `bigquery-public-data.stackoverflow.posts_questions`
    UNION ALL
    SELECT
      user_id AS id,
      creation_date
    FROM
      `bigquery-public-data.stackoverflow.comments`)
  WHERE
    FORMAT_TIMESTAMP('%Y', creation_date) = '2017')
GROUP BY
id
"""
query_job = client.query(query)
results = query_job.result()
activity_count = results.to_dataframe()

In [77]:
with open('./data/processed/activity_count.pkl', 'wb') as picklefile:
    pickle.dump(activity_count, picklefile)

## 2017 Answer count

In [7]:
query = """
#standardSQL
SELECT
id,
count(id) AS count
FROM (
  SELECT
    *
  FROM (
    SELECT
      owner_user_id AS id,
      creation_date
    FROM
      `bigquery-public-data.stackoverflow.posts_answers`)
  WHERE
    FORMAT_TIMESTAMP('%Y', creation_date) = '2017')
GROUP BY
id
"""
query_job = client.query(query)
results = query_job.result()
answer_count = results.to_dataframe()

In [8]:
with open('./data/processed/answer_count.pkl', 'wb') as picklefile:
    pickle.dump(answer_count, picklefile)

## Answer time series 2017

In [12]:
### Construct a timeseries where columns are number of activities per month
query = """
WITH activity_times AS (
SELECT 
  owner_user_id AS id, 
  FORMAT_TIMESTAMP('%Y%m', creation_date) AS creation_time
FROM
  `bigquery-public-data.stackoverflow.posts_answers`
WHERE 
  FORMAT_TIMESTAMP('%Y', creation_date) = '2017'
) SELECT id,
    COUNT(IF(creation_time = "201701", id, NULL)) as m_201701,
    COUNT(IF(creation_time = "201702", id, NULL)) as m_201702,
    COUNT(IF(creation_time = "201703", id, NULL)) as m_201703,
    COUNT(IF(creation_time = "201704", id, NULL)) as m_201704,
    COUNT(IF(creation_time = "201705", id, NULL)) as m_201705,
    COUNT(IF(creation_time = "201706", id, NULL)) as m_201706,
    COUNT(IF(creation_time = "201707", id, NULL)) as m_201707,
    COUNT(IF(creation_time = "201708", id, NULL)) as m_201708,
    COUNT(IF(creation_time = "201709", id, NULL)) as m_201709,
    COUNT(IF(creation_time = "201710", id, NULL)) as m_201710,
    COUNT(IF(creation_time = "201711", id, NULL)) as m_201711,
    COUNT(IF(creation_time = "201712", id, NULL)) as m_201712
  FROM activity_times
  GROUP BY id
  ORDER BY id
"""
query_job = client.query(query)
results = query_job.result()
answer_time_series2 = results.to_dataframe()

In [13]:
answer_time_series2.dropna(inplace=True)
answer_time_series2['id'] = answer_time_series2['id'].astype(int)
with open('./data/processed/answer_time_series2.pkl', 'wb') as picklefile:
    pickle.dump(answer_time_series2, picklefile)

## 2017 Activity timeseries

In [12]:
### Construct a timeseries where columns are number of activities per month
query = """
WITH activity_times AS (
  SELECT id, FORMAT_TIMESTAMP('%Y%m', creation_date) AS creation_time
  FROM(
    SELECT 
      owner_user_id AS id, 
      creation_date
    FROM
      `bigquery-public-data.stackoverflow.posts_answers`
    UNION ALL
    SELECT
      owner_user_id AS id, 
      creation_date
    FROM
      `bigquery-public-data.stackoverflow.posts_questions` UNION ALL
    SELECT
      user_id AS id,
      creation_date
    FROM
      `bigquery-public-data.stackoverflow.comments`)
  WHERE
    FORMAT_TIMESTAMP('%Y', creation_date) = '2017'
) SELECT id,
    COUNT(IF(creation_time = "201701", id, NULL)) as m_201701,
    COUNT(IF(creation_time = "201702", id, NULL)) as m_201702,
    COUNT(IF(creation_time = "201703", id, NULL)) as m_201703,
    COUNT(IF(creation_time = "201704", id, NULL)) as m_201704,
    COUNT(IF(creation_time = "201705", id, NULL)) as m_201705,
    COUNT(IF(creation_time = "201706", id, NULL)) as m_201706,
    COUNT(IF(creation_time = "201707", id, NULL)) as m_201707,
    COUNT(IF(creation_time = "201708", id, NULL)) as m_201708,
    COUNT(IF(creation_time = "201709", id, NULL)) as m_201709,
    COUNT(IF(creation_time = "201710", id, NULL)) as m_201710,
    COUNT(IF(creation_time = "201711", id, NULL)) as m_201711,
    COUNT(IF(creation_time = "201712", id, NULL)) as m_201712
  FROM activity_times
  GROUP BY id
  ORDER BY id
"""
query_job = client.query(query)
results = query_job.result()
activity_time_series = results.to_dataframe()

In [13]:
activity_time_series.dropna(inplace=True)
activity_time_series['id'] = activity_time_series['id'].astype(int)
with open('./data/processed/activity_time_series.pkl', 'wb') as picklefile:
    pickle.dump(activity_time_series, picklefile)

In [5]:
### Construct a timeseries where columns are number of activities per month
query = """
WITH activity_times AS (
  SELECT id, FORMAT_TIMESTAMP('%Y%m', creation_date) AS creation_time
  FROM(
    SELECT 
      owner_user_id AS id, 
      creation_date
    FROM
      `bigquery-public-data.stackoverflow.posts_answers`
    UNION ALL
    SELECT
      owner_user_id AS id, 
      creation_date
    FROM
      `bigquery-public-data.stackoverflow.posts_questions` UNION ALL
    SELECT
      user_id AS id,
      creation_date
    FROM
      `bigquery-public-data.stackoverflow.comments`)
  WHERE
    FORMAT_TIMESTAMP('%Y', creation_date) = '2018'
) SELECT id,
    COUNT(DISTINCT IF(creation_time = "201803", id, NULL)) as m_201803,
    COUNT(DISTINCT IF(creation_time = "201804", id, NULL)) as m_201804,
    COUNT(DISTINCT IF(creation_time = "201805", id, NULL)) as m_201805
  FROM activity_times
  GROUP BY id
  ORDER BY id
"""
query_job = client.query(query)
results = query_job.result()
activity_time_series_2018 = results.to_dataframe()

In [11]:
activity_time_series_2018.dropna(inplace=True)
activity_time_series_2018['id'] = activity_time_series_2018['id'].astype(int)
with open('./data/processed/activity_time_series_2018.pkl', 'wb') as picklefile:
    pickle.dump(activity_time_series_2018, picklefile)

In [5]:
with open('./data/processed/activity_time_series_2018.pkl', 'rb') as picklefile:
    activity_time_series_2018 = pickle.load(picklefile)

In [6]:
activity_time_series_2018

id  m_201803  m_201804  m_201805
1       1        1         1         0       
2       4        1         0         1       
3       13       1         1         1       
4       29       0         0         1       
5       33       1         0         1       
6       49       1         1         0       
7       56       0         1         1       
8       58       0         1         0       
9       62       1         1         1       
10      67       0         1         1       
11      71       0         0         0       
12      76       0         0         0       
13      77       0         0         0       
14      82       0         0         0       
15      83       0         0         0       
16      88       0         1         0       
17      91       1         1         0       
18      95       1         1         0       
19      100      0         0         0       
20      101      0         1         0       
21      106      0         1         0       
22      112      1         1         1       
23      113      0         1         1       
24      116      0         0         1       
25      133      0         1         0       
26      137      0         0         0       
27      147      1         0         0       
28      148      1         0         0       
29      150      1         0         0       
30      152      1         0         0       
...     ...     ..        ..        ..       
690391  9886982  0         0         0       
690392  9886983  0         0         0       
690393  9886998  0         0         0       
690394  9887003  0         0         0       
690395  9887010  0         0         0       
690396  9887026  0         0         0       
690397  9887027  0         0         0       
690398  9887029  0         0         0       
690399  9887033  0         0         0       
690400  9887042  0         0         0       
690401  9887057  0         0         0       
690402  9887059  0         0         0       
690403  9887060  0         0         0       
690404  9887061  0         0         0       
690405  9887063  0         0         0       
690406  9887069  0         0         0       
690407  9887074  0         0         0       
690408  9887077  0         0         0       
690409  9887082  0         0         0       
690410  9887086  0         0         0       
690411  9887097  0         0         0       
690412  9887110  0         0         0       
690413  9887134  0         0         0       
690414  9887143  0         0         0       
690415  9887154  0         0         0       
690416  9887175  0         0         0       
690417  9887200  0         0         0       
690418  9887240  0         0         0       
690419  9887275  0         0         0       
690420  9887324  0         0         0       

[690420 rows x 4 columns]

## Timeseries for answer monthly since 2015

In [34]:
### Construct a timeseries where columns are number of activities per month
query = """
#standardSQL
SELECT
  owner_user_id AS id,
  FORMAT_TIMESTAMP('%Y%m',
    creation_date) AS creation_month,
  count(creation_date) AS answer_count
FROM
  `bigquery-public-data.stackoverflow.posts_answers`
WHERE
  FORMAT_TIMESTAMP('%Y',
    creation_date) > '2015'
GROUP BY
  owner_user_id, FORMAT_TIMESTAMP('%Y%m',
    creation_date)
"""
query_job = client.query(query)
results = query_job.result()
active_months = results.to_dataframe()

In [35]:
active_months.dropna(inplace=True)
active_months['id'] = active_months['id'].astype(int)
with open('./data/processed/active_months.pkl', 'wb') as picklefile:
    pickle.dump(active_months, picklefile)

In [13]:
number_active_months = active_months.groupby('id')['creation_month'].count()
account_morethan_12mon = set(number_active_months[number_active_months>=12].index)

In [22]:
answer_time_series_simple = answer_time_series_simple[answer_time_series_simple.id.isin(account_morethan_12mon)]

## Answer time series 2016 and 2017

In [7]:
### Construct a timeseries where columns are number of activities per month
query = """
#standardsql
WITH answer_times AS (
  SELECT 
    owner_user_id AS id, 
    FORMAT_TIMESTAMP('%Y%m', creation_date) AS creation_time
  FROM
    `bigquery-public-data.stackoverflow.posts_answers`
  WHERE
    FORMAT_TIMESTAMP('%Y', creation_date) = '2017'
    OR
    FORMAT_TIMESTAMP('%Y', creation_date) = '2016'
) SELECT id,
    COUNT(IF(creation_time = "201601", id, NULL)) as m_201601,
    COUNT(IF(creation_time = "201602", id, NULL)) as m_201602,
    COUNT(IF(creation_time = "201603", id, NULL)) as m_201603,
    COUNT(IF(creation_time = "201604", id, NULL)) as m_201604,
    COUNT(IF(creation_time = "201605", id, NULL)) as m_201605,
    COUNT(IF(creation_time = "201606", id, NULL)) as m_201606,
    COUNT(IF(creation_time = "201607", id, NULL)) as m_201607,
    COUNT(IF(creation_time = "201608", id, NULL)) as m_201608,
    COUNT(IF(creation_time = "201609", id, NULL)) as m_201609,
    COUNT(IF(creation_time = "201610", id, NULL)) as m_201610,
    COUNT(IF(creation_time = "201611", id, NULL)) as m_201611,
    COUNT(IF(creation_time = "201612", id, NULL)) as m_201612,
    COUNT(IF(creation_time = "201701", id, NULL)) as m_201701,
    COUNT(IF(creation_time = "201702", id, NULL)) as m_201702,
    COUNT(IF(creation_time = "201703", id, NULL)) as m_201703,
    COUNT(IF(creation_time = "201704", id, NULL)) as m_201704,
    COUNT(IF(creation_time = "201705", id, NULL)) as m_201705,
    COUNT(IF(creation_time = "201706", id, NULL)) as m_201706,
    COUNT(IF(creation_time = "201707", id, NULL)) as m_201707,
    COUNT(IF(creation_time = "201708", id, NULL)) as m_201708,
    COUNT(IF(creation_time = "201709", id, NULL)) as m_201709,
    COUNT(IF(creation_time = "201710", id, NULL)) as m_201710,
    COUNT(IF(creation_time = "201711", id, NULL)) as m_201711,
    COUNT(IF(creation_time = "201712", id, NULL)) as m_201712
  FROM answer_times
  GROUP BY id
  ORDER BY id
"""
query_job = client.query(query)
results = query_job.result()
answer_time_series_2years = results.to_dataframe()

In [8]:
answer_time_series_2years.dropna(inplace=True)
answer_time_series_2years['id'] = answer_time_series_2years['id'].astype(int)
with open('./data/processed/answer_time_series_2years2.pkl', 'wb') as picklefile:
    pickle.dump(answer_time_series_2years, picklefile)

### Retrieve basic user stats

In [5]:
## Create a view to reconstruct user reputation at 01/01/2017 and 01/01/2018
query = """
#standardSQL
SELECT
  id,
  LENGTH(about_me) AS about_me_length,
  CAST(FORMAT_TIMESTAMP('%Y',
      creation_date) AS int64) AS creation_year,
  CAST(FORMAT_TIMESTAMP('%m',
      creation_date) AS int64) AS creation_month,
  CAST(FORMAT_TIMESTAMP('%Y',
      last_access_date) AS int64) AS last_access_year,
  CAST(FORMAT_TIMESTAMP('%m',
      last_access_date) AS int64) AS last_access_month,
  location,
  up_votes,
  down_votes,
  CASE
    WHEN profile_image_url = '' THEN 1
    ELSE 0
  END AS profile_image,
  website_url
FROM
  `bigquery-public-data.stackoverflow.users`
WHERE
  CAST(FORMAT_TIMESTAMP('%Y',
      creation_date) AS int64) < 2018
"""
query_job = client.query(query)
results = query_job.result()
user_basic = results.to_dataframe()

In [6]:
with open('./data/processed/user_basic.pkl', 'wb') as picklefile:
    pickle.dump(user_basic, picklefile)

In [19]:
with open('./data/processed/user_basic.pkl', 'rb') as picklefile:
    user_basic = pickle.load(picklefile)

In [21]:
user_basic.set_index('id', inplace=True)

## Answer count year over year

In [4]:
query = """
#standardsql
SELECT 
CAST(FORMAT_TIMESTAMP('%Y%m', creation_date) AS int64) AS creation_month,
COUNT(creation_date) AS answer_count
FROM
`bigquery-public-data.stackoverflow.posts_answers`
WHERE
CAST(FORMAT_TIMESTAMP('%Y', creation_date) AS int64) > 2012 AND
CAST(FORMAT_TIMESTAMP('%Y', creation_date) AS int64) < 2018
GROUP BY 
CAST(FORMAT_TIMESTAMP('%Y%m', creation_date) AS int64)
"""
query_job = client.query(query)
results = query_job.result()
answer_count_trend = results.to_dataframe()

In [6]:
query = """
#standardsql
SELECT 
CAST(FORMAT_TIMESTAMP('%Y%m', creation_date) AS int64) AS creation_month,
COUNT(creation_date) AS question_count
FROM
`bigquery-public-data.stackoverflow.posts_questions`
WHERE
CAST(FORMAT_TIMESTAMP('%Y', creation_date) AS int64) > 2012 AND
CAST(FORMAT_TIMESTAMP('%Y', creation_date) AS int64) < 2018
GROUP BY 
CAST(FORMAT_TIMESTAMP('%Y%m', creation_date) AS int64)
"""
query_job = client.query(query)
results = query_job.result()
question_count_trend = results.to_dataframe()

In [23]:
query = """
#standardsql
SELECT 
CAST(FORMAT_TIMESTAMP('%Y%m', creation_date) AS int64) AS creation_month,
COUNT(creation_date) AS comment_count
FROM
`bigquery-public-data.stackoverflow.comments`
WHERE
CAST(FORMAT_TIMESTAMP('%Y', creation_date) AS int64) > 2012 AND
CAST(FORMAT_TIMESTAMP('%Y', creation_date) AS int64) < 2018
GROUP BY 
CAST(FORMAT_TIMESTAMP('%Y%m', creation_date) AS int64)
"""
query_job = client.query(query)
results = query_job.result()
comment_count_trend = results.to_dataframe()

In [24]:
answer_count_trend.set_index('creation_month', inplace=True)
question_count_trend.set_index('creation_month', inplace=True)
comment_count_trend.set_index('creation_month', inplace=True)

In [25]:
all_activity_count = answer_count_trend.join(question_count_trend)
all_activity_count = all_activity_count.join(comment_count_trend)

In [30]:
all_activity_count.reset_index(inplace=True)
all_activity_count.sort_values('creation_month', inplace=True)
all_activity_count['creation_month'] = pd.to_datetime(all_activity_count['creation_month'], format='%Y%m')

In [34]:
all_activity_count.drop('index', axis=1, inplace=True)

In [82]:
answer_count_trend['answer_count'] = answer_count_trend['answer_count']*1000

In [41]:
def y_fmt(x, y):
    return '{:2.1e}'.format(x)

fig = plt.figure(figsize=(16, 8), dpi=80)
ax = fig.add_subplot(111)
plt.plot(all_activity_count.creation_month, all_activity_count.answer_count, label='No. of answers')
plt.plot(all_activity_count.creation_month, all_activity_count.question_count, label='No. of questions')
#plt.plot(all_activity_count.creation_month, all_activity_count.comment_count , label='No. of comments')
plt.xlabel('Year', fontsize=20)
plt.ylabel('Number of posts', fontsize=20)
plt.title('Monthly activity count on Stack Overflow', fontsize=30)
plt.legend()
ax.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(y_fmt))
plt.show()
fig.savefig('./data/img/Monthly_Activity.png')

In [15]:
answer_count_trend['creation_year'] = answer_count_trend['creation_month'].apply(lambda x:x[:4])

In [20]:
answer_count_last5years = answer_count_trend[(answer_count_trend['creation_year'].astype(int)>2012) & (answer_count_trend['creation_year'].astype(int)<2018)]

In [21]:
answer_count_last5years.groupby('creation_year')

creation_month  answer_count  year creation_year
23   201301         267308        2013  2013        
2    201302         256454        2013  2013        
9    201303         290754        2013  2013        
48   201304         283059        2013  2013        
13   201305         275362        2013  2013        
26   201306         258740        2013  2013        
54   201307         289300        2013  2013        
70   201308         284174        2013  2013        
110  201309         272206        2013  2013        
93   201310         301127        2013  2013        
60   201311         284679        2013  2013        
1    201312         273933        2013  2013        
57   201401         309546        2014  2014        
10   201402         299863        2014  2014        
59   201403         320271        2014  2014        
92   201404         296191        2014  2014        
63   201405         265005        2014  2014        
37   201406         239698        2014  2014        
108  201407         264041        2014  2014        
86   201408         244608        2014  2014        
18   201409         245896        2014  2014        
17   201410         253322        2014  2014        
94   201411         244768        2014  2014        
4    201412         233579        2014  2014        
61   201501         243927        2015  2015        
55   201502         244721        2015  2015        
107  201503         271039        2015  2015        
88   201504         283760        2015  2015        
120  201505         274443        2015  2015        
75   201506         272645        2015  2015        
20   201507         284068        2015  2015        
12   201508         263812        2015  2015        
118  201509         258046        2015  2015        
90   201510         266510        2015  2015        
111  201511         251915        2015  2015        
101  201512         250081        2015  2015        
112  201601         263106        2016  2016        
58   201602         269967        2016  2016        
14   201603         287568        2016  2016        
16   201604         279024        2016  2016        
97   201605         270596        2016  2016        
56   201606         260123        2016  2016        
67   201607         249837        2016  2016        
73   201608         257443        2016  2016        
84   201609         242816        2016  2016        
87   201610         245289        2016  2016        
50   201611         242736        2016  2016        
8    201612         229962        2016  2016        
109  201701         251845        2017  2017        
113  201702         248161        2017  2017        
30   201703         282220        2017  2017        
38   201704         247705        2017  2017        
91   201705         259651        2017  2017        
100  201706         248402        2017  2017        
40   201707         254754        2017  2017        
15   201708         252924        2017  2017        
6    201709         227524        2017  2017        
34   201710         234943        2017  2017        
39   201711         232428        2017  2017        
33   201712         199725        2017  2017

## Corhort analysis

In [46]:
view_ref = dataset_ref.table('SO_cohort_items_2017_view')
view = bigquery.Table(view_ref)

view.view_query = """
#standardSQL
SELECT
  id,
  FORMAT_TIMESTAMP('%Y%m', creation_date) AS signup_month
FROM
  `bigquery-public-data.stackoverflow.users`
WHERE
  CAST(FORMAT_TIMESTAMP('%Y', creation_date) AS int64) = 2017
ORDER BY
  id,
  FORMAT_TIMESTAMP('%Y%m', creation_date)
"""
view = client.create_table(view)

In [42]:
view_ref = dataset_ref.table('SO_answerer_activity_2017_view')
view = bigquery.Table(view_ref)

view.view_query = """
#standardSQL
SELECT
answer.owner_user_id AS id,
(CAST(FORMAT_TIMESTAMP('%m',
  answer.creation_date) AS int64) - cohort_items.signup_month) AS month_number
FROM
`bigquery-public-data.stackoverflow.posts_answers` AS answer
LEFT JOIN
`stackoverflow-metis.SO_dataset.SO_cohort_items_2017_view` AS cohort_items
ON
answer.id = cohort_items.id
WHERE
CAST(FORMAT_TIMESTAMP('%Y',
      answer.creation_date) AS int64) = 2017
GROUP BY
1,
2
"""
view = client.create_table(view)

In [48]:
view_ref = dataset_ref.table('SO_answerer_month_number_2017_view')
view = bigquery.Table(view_ref)

view.view_query = """
#standardSQL
WITH activity_signup AS(
SELECT
  answer.owner_user_id AS id,
  CAST(FORMAT_TIMESTAMP('%m', answer.creation_date) AS int64) AS activity_month,
  MIN(CAST(SUBSTR(cohort_items.signup_month, 5,6) AS int64)) AS signup_month
FROM
  `bigquery-public-data.stackoverflow.posts_answers` AS answer
LEFT JOIN
  `stackoverflow-metis.SO_dataset.SO_cohort_items_2017_view` AS cohort_items
ON
  answer.owner_user_id = cohort_items.id
WHERE
  CAST(FORMAT_TIMESTAMP('%Y', answer.creation_date) AS int64) = 2017
GROUP BY
1,2) SELECT DISTINCT id, (activity_month - signup_month) AS month_number
FROM activity_signup
WHERE signup_month is not NULL
"""
view = client.create_table(view)

In [49]:
query = """
#standardSQL
WITH
  retention_table AS (
  SELECT
    cohort_items.signup_month,
    user_activities.month_number,
    COUNT(1) AS num_users
  FROM
    `stackoverflow-metis.SO_dataset.SO_answerer_month_number_2017_view` AS user_activities
  LEFT JOIN
    `stackoverflow-metis.SO_dataset.SO_cohort_items_2017_view` AS cohort_items
  ON
    user_activities.id = cohort_items.id
  GROUP BY
    1,
    2 )
SELECT
  retention_table.signup_month,
  cohort_size.num_users AS total_users,
  retention_table.month_number,
  retention_table.num_users*100/ cohort_size.num_users AS percentage
FROM
  retention_table
LEFT JOIN (
  SELECT
    signup_month AS cohort_month,
    COUNT(signup_month) AS num_users
  FROM
    `stackoverflow-metis.SO_dataset.SO_cohort_items_2017_view`
  GROUP BY
    signup_month
  ORDER BY
    signup_month) AS cohort_size
ON
  cohort_size.cohort_month = retention_table.signup_month
WHERE
  retention_table.signup_month IS NOT NULL
ORDER BY
  1,
  3
  """
query_job = client.query(query)
results = query_job.result()
corhort_analysis = results.to_dataframe()

In [52]:
corhort_analysis.to_csv('./data/processed/cohort.analysis.csv')

## Cohort Analysis by Year

In [53]:
view_ref = dataset_ref.table('SO_cohort_items_view')
view = bigquery.Table(view_ref)

view.view_query = """
#standardSQL
SELECT
  id,
  CAST(FORMAT_TIMESTAMP('%Y', creation_date) AS int64) AS signup_year
FROM
  `bigquery-public-data.stackoverflow.users`
WHERE
  CAST(FORMAT_TIMESTAMP('%Y', creation_date) AS int64) >= 2008 AND
  CAST(FORMAT_TIMESTAMP('%Y', creation_date) AS int64) < 2018
ORDER BY
  1,2
"""
view = client.create_table(view)

In [54]:
view_ref = dataset_ref.table('SO_answerer_year_number_view')
view = bigquery.Table(view_ref)

view.view_query = """
#standardSQL
WITH
  activity_signup AS(
  SELECT
    answer.owner_user_id AS id,
    CAST(FORMAT_TIMESTAMP('%Y', answer.creation_date) AS int64) AS activity_year,
    MIN(cohort_items.signup_year) AS signup_year
  FROM
    `bigquery-public-data.stackoverflow.posts_answers` AS answer
  LEFT JOIN
    `stackoverflow-metis.SO_dataset.SO_cohort_items_view` AS cohort_items
  ON
    answer.owner_user_id = cohort_items.id
  WHERE
      CAST(FORMAT_TIMESTAMP('%Y', answer.creation_date) AS int64) >= 2008 AND
      CAST(FORMAT_TIMESTAMP('%Y', answer.creation_date) AS int64) < 2018
  GROUP BY
    1,
    2)
SELECT
  DISTINCT id,
  (activity_year - signup_year) AS year_number
FROM
  activity_signup
WHERE
  signup_year IS NOT NULL
"""
view = client.create_table(view)

In [55]:
query = """
#standardSQL
WITH
  retention_table AS (
  SELECT
    cohort_items.signup_year,
    user_activities.year_number,
    COUNT(1) AS num_users
  FROM
    `stackoverflow-metis.SO_dataset.SO_answerer_year_number_view` AS user_activities
  LEFT JOIN
    `stackoverflow-metis.SO_dataset.SO_cohort_items_view` AS cohort_items
  ON
    user_activities.id = cohort_items.id
  GROUP BY
    1,
    2 )
SELECT
  retention_table.signup_year,
  cohort_size.num_users AS total_users,
  retention_table.year_number,
  retention_table.num_users*100/ cohort_size.num_users AS percentage
FROM
  retention_table
LEFT JOIN (
  SELECT
    signup_year AS cohort_year,
    COUNT(signup_year) AS num_users
  FROM
    `stackoverflow-metis.SO_dataset.SO_cohort_items_view`
  GROUP BY
    signup_year
  ORDER BY
    signup_year) AS cohort_size
ON
  cohort_size.cohort_year = retention_table.signup_year
WHERE
  retention_table.signup_year IS NOT NULL
ORDER BY
  1,
  3
  """
query_job = client.query(query)
results = query_job.result()
corhort_analysis_year = results.to_dataframe()

In [57]:
corhort_analysis_year.to_csv('./data/processed/cohort_analysis_year.csv')

## LCV analysis for customers registered between 2016 Jan and June

### Reconstruct user reputation

In [50]:
query = """
#standardSQL
SELECT
  id
FROM 
  `bigquery-public-data.stackoverflow.users` AS user
WHERE
  CAST(FORMAT_TIMESTAMP('%Y%m', user.creation_date) AS int64) >= 201601
  AND CAST(FORMAT_TIMESTAMP('%Y%m', user.creation_date) AS int64) <= 201606

"""
query_job = client.query(query)
results = query_job.result()
lcv_id = results.to_dataframe()

lcv_id.dropna(inplace=True)
lcv_id['id'] = lcv_accepted['id'].astype(int)
lcv_id.sort_values('id',inplace=True)

with open('./data/processed/lcv_id.pkl', 'wb') as picklefile:
    pickle.dump(lcv_id, picklefile)

In [42]:
### Construct a timeseries where columns are number of activities per month
query = """
#standardsql
WITH
  answer_times AS (
  SELECT
    owner_user_id AS id,
    score,
    FORMAT_TIMESTAMP('%Y%m', creation_date) AS creation_time
  FROM
    `bigquery-public-data.stackoverflow.posts_answers`
  WHERE
    FORMAT_TIMESTAMP('%Y', creation_date) = '2016'
    OR FORMAT_TIMESTAMP('%Y', creation_date) = '2017'
    OR FORMAT_TIMESTAMP('%Y', creation_date) = '2018' )
SELECT
  id,
  SUM(IF(creation_time = "201601", score, NULL)) as m_201601,
  SUM(IF(creation_time = "201602", score, NULL)) as m_201602,
  SUM(IF(creation_time = "201603", score, NULL)) as m_201603,
  SUM(IF(creation_time = "201604", score, NULL)) as m_201604,
  SUM(IF(creation_time = "201605", score, NULL)) as m_201605,
  SUM(IF(creation_time = "201606", score, NULL)) as m_201606,
  SUM(IF(creation_time = "201607", score, NULL)) as m_201607,
  SUM(IF(creation_time = "201608", score, NULL)) as m_201608,
  SUM(IF(creation_time = "201609", score, NULL)) as m_201609,
  SUM(IF(creation_time = "201610", score, NULL)) as m_201610,
  SUM(IF(creation_time = "201611", score, NULL)) as m_201611,
  SUM(IF(creation_time = "201612", score, NULL)) as m_201612,
  SUM(IF(creation_time = "201701", score, NULL)) as m_201701,
  SUM(IF(creation_time = "201702", score, NULL)) as m_201702,
  SUM(IF(creation_time = "201703", score, NULL)) as m_201703,
  SUM(IF(creation_time = "201704", score, NULL)) as m_201704,
  SUM(IF(creation_time = "201705", score, NULL)) as m_201705,
  SUM(IF(creation_time = "201706", score, NULL)) as m_201706,
  SUM(IF(creation_time = "201707", score, NULL)) as m_201707,
  SUM(IF(creation_time = "201708", score, NULL)) as m_201708,
  SUM(IF(creation_time = "201709", score, NULL)) as m_201709,
  SUM(IF(creation_time = "201710", score, NULL)) as m_201710,
  SUM(IF(creation_time = "201711", score, NULL)) as m_201711,
  SUM(IF(creation_time = "201712", score, NULL)) as m_201712,
  SUM(IF(creation_time = "201801", score, NULL)) as m_201801,
  SUM(IF(creation_time = "201802", score, NULL)) as m_201802,
  SUM(IF(creation_time = "201803", score, NULL)) as m_201803,
  SUM(IF(creation_time = "201804", score, NULL)) as m_201804,
  SUM(IF(creation_time = "201805", score, NULL)) as m_201805,
  SUM(IF(creation_time = "201806", score, NULL)) as m_201806
FROM
  answer_times
WHERE
  id IN (
  SELECT
    id
  FROM
    `bigquery-public-data.stackoverflow.users` AS user
  WHERE
    CAST(FORMAT_TIMESTAMP('%Y%m', user.creation_date) AS int64) >= 201601
    AND CAST(FORMAT_TIMESTAMP('%Y%m', user.creation_date) AS int64) <= 201606)
GROUP BY
  id
ORDER BY
  id
"""
query_job = client.query(query)
results = query_job.result()
lcv_answer = results.to_dataframe()

with open('./data/processed/lcv_answer.pkl', 'wb') as picklefile:
    pickle.dump(lcv_answer, picklefile)

In [44]:
lcv_answer.fillna(0,inplace=True)

In [46]:
lcv_answer.head()

id  m_201601  m_201602  m_201603  m_201604  m_201605  m_201606  \
0  5735120  0.0       0.0       1.0       0.0       0.0       0.0        
1  5735122  0.0       0.0       0.0       0.0       0.0       0.0        
2  5735124 -1.0       0.0       0.0       0.0       0.0       1.0        
3  5735139  9.0       0.0       0.0       0.0       0.0       0.0        
4  5735144  5.0       0.0       0.0       0.0       0.0       0.0        

   m_201607  m_201608  m_201609    ...     m_201709  m_201710  m_201711  \
0  0.0       0.0       0.0         ...     0.0       0.0       0.0        
1  0.0       0.0       0.0         ...     0.0       0.0       0.0        
2  0.0       0.0       0.0         ...     0.0       0.0       0.0        
3  0.0       0.0       0.0         ...     0.0       0.0       0.0        
4  0.0       0.0       0.0         ...     0.0       0.0       0.0        

   m_201712  m_201801  m_201802  m_201803  m_201804  m_201805  m_201806  
0  0.0       0.0       0.0       0.0       0.0       0.0       0.0       
1  0.0       0.0       0.0       0.0       0.0       0.0       0.0       
2  0.0       0.0       0.0       0.0       0.0       0.0       0.0       
3  0.0       0.0       0.0       0.0       0.0       0.0       0.0       
4  0.0       0.0       0.0       0.0       0.0       0.0       0.0       

[5 rows x 31 columns]

In [8]:
### Construct a timeseries where columns are number of activities per month
query = """
#standardsql
#standardSQL
WITH
  question_times AS (
  SELECT
    owner_user_id AS id,
    FORMAT_TIMESTAMP('%Y%m', creation_date) AS creation_time
  FROM
    `bigquery-public-data.stackoverflow.posts_questions`
  WHERE
    FORMAT_TIMESTAMP('%Y', creation_date) = '2016'
    OR FORMAT_TIMESTAMP('%Y', creation_date) = '2017'
    OR FORMAT_TIMESTAMP('%Y', creation_date) = '2018' )
SELECT
  id,
  COUNT(IF(creation_time = "201601", id, NULL)) as m_201601,
  COUNT(IF(creation_time = "201602", id, NULL)) as m_201602,
  COUNT(IF(creation_time = "201603", id, NULL)) as m_201603,
  COUNT(IF(creation_time = "201604", id, NULL)) as m_201604,
  COUNT(IF(creation_time = "201605", id, NULL)) as m_201605,
  COUNT(IF(creation_time = "201606", id, NULL)) as m_201606,
  COUNT(IF(creation_time = "201607", id, NULL)) as m_201607,
  COUNT(IF(creation_time = "201608", id, NULL)) as m_201608,
  COUNT(IF(creation_time = "201609", id, NULL)) as m_201609,
  COUNT(IF(creation_time = "201610", id, NULL)) as m_201610,
  COUNT(IF(creation_time = "201611", id, NULL)) as m_201611,
  COUNT(IF(creation_time = "201612", id, NULL)) as m_201612,
  COUNT(IF(creation_time = "201701", id, NULL)) as m_201701,
  COUNT(IF(creation_time = "201702", id, NULL)) as m_201702,
  COUNT(IF(creation_time = "201703", id, NULL)) as m_201703,
  COUNT(IF(creation_time = "201704", id, NULL)) as m_201704,
  COUNT(IF(creation_time = "201705", id, NULL)) as m_201705,
  COUNT(IF(creation_time = "201706", id, NULL)) as m_201706,
  COUNT(IF(creation_time = "201707", id, NULL)) as m_201707,
  COUNT(IF(creation_time = "201708", id, NULL)) as m_201708,
  COUNT(IF(creation_time = "201709", id, NULL)) as m_201709,
  COUNT(IF(creation_time = "201710", id, NULL)) as m_201710,
  COUNT(IF(creation_time = "201711", id, NULL)) as m_201711,
  COUNT(IF(creation_time = "201712", id, NULL)) as m_201712,
  COUNT(IF(creation_time = "201801", id, NULL)) as m_201801,
  COUNT(IF(creation_time = "201802", id, NULL)) as m_201802,
  COUNT(IF(creation_time = "201803", id, NULL)) as m_201803,
  COUNT(IF(creation_time = "201804", id, NULL)) as m_201804,
  COUNT(IF(creation_time = "201805", id, NULL)) as m_201805,
  COUNT(IF(creation_time = "201806", id, NULL)) as m_201806
FROM
  question_times
WHERE
  id IN (
  SELECT
    id
  FROM
    `bigquery-public-data.stackoverflow.users` AS user
  WHERE
    CAST(FORMAT_TIMESTAMP('%Y%m', user.creation_date) AS int64) >= 201601
    AND CAST(FORMAT_TIMESTAMP('%Y%m', user.creation_date) AS int64) <= 201606)
GROUP BY
  id
ORDER BY
  id
"""
query_job = client.query(query)
results = query_job.result()
lcv_question = results.to_dataframe()

with open('./data/processed/lcv_question.pkl', 'wb') as picklefile:
    pickle.dump(lcv_answer, picklefile)

In [48]:
### Construct a timeseries where columns are number of activities per month
query = """
#standardsql
WITH
  question_times AS (
  SELECT
    owner_user_id AS id,
    score,
    FORMAT_TIMESTAMP('%Y%m', creation_date) AS creation_time
  FROM
    `bigquery-public-data.stackoverflow.posts_questions`
  WHERE
    FORMAT_TIMESTAMP('%Y', creation_date) = '2016'
    OR FORMAT_TIMESTAMP('%Y', creation_date) = '2017'
    OR FORMAT_TIMESTAMP('%Y', creation_date) = '2018' )
SELECT
  id,
  SUM(IF(creation_time = "201601", score, NULL)) as m_201601,
  SUM(IF(creation_time = "201602", score, NULL)) as m_201602,
  SUM(IF(creation_time = "201603", score, NULL)) as m_201603,
  SUM(IF(creation_time = "201604", score, NULL)) as m_201604,
  SUM(IF(creation_time = "201605", score, NULL)) as m_201605,
  SUM(IF(creation_time = "201606", score, NULL)) as m_201606,
  SUM(IF(creation_time = "201607", score, NULL)) as m_201607,
  SUM(IF(creation_time = "201608", score, NULL)) as m_201608,
  SUM(IF(creation_time = "201609", score, NULL)) as m_201609,
  SUM(IF(creation_time = "201610", score, NULL)) as m_201610,
  SUM(IF(creation_time = "201611", score, NULL)) as m_201611,
  SUM(IF(creation_time = "201612", score, NULL)) as m_201612,
  SUM(IF(creation_time = "201701", score, NULL)) as m_201701,
  SUM(IF(creation_time = "201702", score, NULL)) as m_201702,
  SUM(IF(creation_time = "201703", score, NULL)) as m_201703,
  SUM(IF(creation_time = "201704", score, NULL)) as m_201704,
  SUM(IF(creation_time = "201705", score, NULL)) as m_201705,
  SUM(IF(creation_time = "201706", score, NULL)) as m_201706,
  SUM(IF(creation_time = "201707", score, NULL)) as m_201707,
  SUM(IF(creation_time = "201708", score, NULL)) as m_201708,
  SUM(IF(creation_time = "201709", score, NULL)) as m_201709,
  SUM(IF(creation_time = "201710", score, NULL)) as m_201710,
  SUM(IF(creation_time = "201711", score, NULL)) as m_201711,
  SUM(IF(creation_time = "201712", score, NULL)) as m_201712,
  SUM(IF(creation_time = "201801", score, NULL)) as m_201801,
  SUM(IF(creation_time = "201802", score, NULL)) as m_201802,
  SUM(IF(creation_time = "201803", score, NULL)) as m_201803,
  SUM(IF(creation_time = "201804", score, NULL)) as m_201804,
  SUM(IF(creation_time = "201805", score, NULL)) as m_201805,
  SUM(IF(creation_time = "201806", score, NULL)) as m_201806
FROM
  question_times
WHERE
  id IN (
  SELECT
    id
  FROM
    `bigquery-public-data.stackoverflow.users` AS user
  WHERE
    CAST(FORMAT_TIMESTAMP('%Y%m', user.creation_date) AS int64) >= 201601
    AND CAST(FORMAT_TIMESTAMP('%Y%m', user.creation_date) AS int64) <= 201606)
GROUP BY
  id
ORDER BY
  id
"""
query_job = client.query(query)
results = query_job.result()
lcv_question = results.to_dataframe()



In [49]:
lcv_question.fillna(0)


id  m_201601  m_201602  m_201603  m_201604  m_201605  m_201606  \
0  5735115 NaN       NaN       NaN       NaN       NaN       NaN         
1  5735116  0.0      NaN       NaN       NaN       NaN       NaN         
2  5735123 -1.0      NaN       NaN       NaN       NaN       NaN         
3  5735124 NaN       NaN       NaN        0.0      NaN        1.0        
4  5735127  0.0       0.0      NaN       NaN       NaN       NaN         

   m_201607  m_201608  m_201609    ...     m_201709  m_201710  m_201711  \
0 NaN       NaN       NaN          ...    NaN       NaN       NaN         
1 NaN       NaN       NaN          ...    NaN       NaN       NaN         
2 NaN       NaN       NaN          ...    NaN       NaN       NaN         
3 NaN       NaN       NaN          ...     2.0       2.0      NaN         
4 NaN       NaN       NaN          ...    NaN       NaN       NaN         

   m_201712  m_201801  m_201802  m_201803  m_201804  m_201805  m_201806  
0 NaN       NaN       NaN       NaN       NaN       NaN       NaN        
1 NaN       NaN       NaN       NaN       NaN       NaN       NaN        
2 NaN       NaN       NaN       NaN       NaN       NaN       NaN        
3  0.0      NaN       NaN       NaN       NaN       NaN       NaN        
4 NaN       NaN       NaN       NaN       NaN       NaN       NaN        

[5 rows x 31 columns]

In [23]:
lcv_question.set_index('id', inplace=True)
test = lcv_question.join(user_basic, how='left')

In [25]:
test.tail(100)

m_201601  m_201602  m_201603  m_201604  m_201605  m_201606  \
id                                                                     
7173333   0         0         0         0         0         0          
7190044   0         0         0         0         0         0          
7193224   0         0         0         0         0         0          
7205111   0         0         0         0         1         0          
7232732   0         0         5         0         0         0          
7239355   0         0         0         0         0         2          
7241614   0         0         0         0         0         0          
7249589   0         0         0         0         0         0          
7272437   0         0         0         0         0         0          
7290169   0         0         0         0         0         0          
7303739   0         0         0         0         0         0          
7312203   0         0         0         0         1         0          
7320067   0         1         0         0         0         0          
7331567   0         0         0         0         0         0          
7341447   0         0         0         0         0         0          
7365350   0         0         0         0         0         0          
7377262   0         0         0         0         0         0          
7456874   0         0         0         0         0         0          
7463809   0         0         0         0         0         0          
7490099   0         0         0         0         0         0          
7506213   0         0         0         1         1         0          
7555897   0         0         0         0         0         0          
7585104   0         0         0         0         0         0          
7596472   0         0         0         0         0         0          
7600596   0         1         0         0         0         0          
7602380   0         0         0         1         0         0          
7605234   0         0         0         0         0         0          
7619211   0         0         0         0         0         0          
7652465   0         0         0         0         0         0          
7653476   0         0         0         0         0         0          
...      ..        ..        ..        ..        ..        ..          
8797268   3         1         0         0         0         0          
8871635   0         0         0         0         0         1          
8889105   0         0         0         0         0         0          
8932976   0         0         0         0         0         0          
8971365   0         0         1         0         0         1          
9032203   0         0         0         0         0         0          
9038547   0         0         0         0         0         1          
9062087   0         0         0         0         0         0          
9086856   0         0         0         0         1         0          
9147974   0         0         0         0         0         0          
9229539   0         0         0         0         0         0          
9257724   0         0         0         0         0         0          
9264839   0         0         0         0         0         3          
9335590   0         0         0         0         0         1          
9439161   0         0         0         1         0         0          
9481513   0         0         1         0         0         0          
9485608   0         0         0         0         0         0          
9608077   0         0         0         0         0         0          
9613823   0         0         0         0         0         1          
9636650   0         0         0         0         0         0          
9645203   0         0         0         0         0         0          
9655834   0         0         0         0         1         0          
9677832   0         0         0         0         

In [26]:
query = """
#standardSQL
WITH
  accepted_answer AS (
  SELECT
    answer.owner_user_id AS id,
    FORMAT_TIMESTAMP('%Y%m', answer.creation_date) AS creation_time
  FROM
    `bigquery-public-data.stackoverflow.posts_answers` AS answer
  LEFT JOIN
    `bigquery-public-data.stackoverflow.posts_questions` AS question
  ON
    answer.parent_id = question.id
  WHERE
    answer.id = question.accepted_answer_id
    AND FORMAT_TIMESTAMP('%Y', answer.creation_date) = '2016'
    OR FORMAT_TIMESTAMP('%Y', answer.creation_date) = '2017'
    OR FORMAT_TIMESTAMP('%Y', answer.creation_date) = '2018')
SELECT
  accepted_answer.id,
  COUNT(IF(creation_time = "201601", accepted_answer.id, NULL)) as m_201601,
  COUNT(IF(creation_time = "201602", accepted_answer.id, NULL)) as m_201602,
  COUNT(IF(creation_time = "201603", accepted_answer.id, NULL)) as m_201603,
  COUNT(IF(creation_time = "201604", accepted_answer.id, NULL)) as m_201604,
  COUNT(IF(creation_time = "201605", accepted_answer.id, NULL)) as m_201605,
  COUNT(IF(creation_time = "201606", accepted_answer.id, NULL)) as m_201606,
  COUNT(IF(creation_time = "201607", accepted_answer.id, NULL)) as m_201607,
  COUNT(IF(creation_time = "201608", accepted_answer.id, NULL)) as m_201608,
  COUNT(IF(creation_time = "201609", accepted_answer.id, NULL)) as m_201609,
  COUNT(IF(creation_time = "201610", accepted_answer.id, NULL)) as m_201610,
  COUNT(IF(creation_time = "201611", accepted_answer.id, NULL)) as m_201611,
  COUNT(IF(creation_time = "201612", accepted_answer.id, NULL)) as m_201612,
  COUNT(IF(creation_time = "201701", accepted_answer.id, NULL)) as m_201701,
  COUNT(IF(creation_time = "201702", accepted_answer.id, NULL)) as m_201702,
  COUNT(IF(creation_time = "201703", accepted_answer.id, NULL)) as m_201703,
  COUNT(IF(creation_time = "201704", accepted_answer.id, NULL)) as m_201704,
  COUNT(IF(creation_time = "201705", accepted_answer.id, NULL)) as m_201705,
  COUNT(IF(creation_time = "201706", accepted_answer.id, NULL)) as m_201706,
  COUNT(IF(creation_time = "201707", accepted_answer.id, NULL)) as m_201707,
  COUNT(IF(creation_time = "201708", accepted_answer.id, NULL)) as m_201708,
  COUNT(IF(creation_time = "201709", accepted_answer.id, NULL)) as m_201709,
  COUNT(IF(creation_time = "201710", accepted_answer.id, NULL)) as m_201710,
  COUNT(IF(creation_time = "201711", accepted_answer.id, NULL)) as m_201711,
  COUNT(IF(creation_time = "201712", accepted_answer.id, NULL)) as m_201712,
  COUNT(IF(creation_time = "201801", accepted_answer.id, NULL)) as m_201801,
  COUNT(IF(creation_time = "201802", accepted_answer.id, NULL)) as m_201802,
  COUNT(IF(creation_time = "201803", accepted_answer.id, NULL)) as m_201803,
  COUNT(IF(creation_time = "201804", accepted_answer.id, NULL)) as m_201804,
  COUNT(IF(creation_time = "201805", accepted_answer.id, NULL)) as m_201805,
  COUNT(IF(creation_time = "201806", accepted_answer.id, NULL)) as m_201806
FROM
  accepted_answer
RIGHT JOIN
  `bigquery-public-data.stackoverflow.users` AS user
ON
  user.id = accepted_answer.id AND
  CAST(FORMAT_TIMESTAMP('%Y%m', user.creation_date) AS int64) >= 201601
  AND CAST(FORMAT_TIMESTAMP('%Y%m', user.creation_date) AS int64) <= 201606
GROUP BY
  accepted_answer.id
ORDER BY
  accepted_answer.id
"""
query_job = client.query(query)
results = query_job.result()
lcv_accepted = results.to_dataframe()

lcv_accepted.dropna(inplace=True)
lcv_accepted['id'] = lcv_accepted['id'].astype(int)

with open('./data/processed/lcv_accepted.pkl', 'wb') as picklefile:
    pickle.dump(lcv_accepted, picklefile)

In [27]:
lcv_accepted.shape

(70831, 31)

In [29]:
query = """
#standardSQL
SELECT
  id
FROM 
  `bigquery-public-data.stackoverflow.users` AS user
WHERE
  CAST(FORMAT_TIMESTAMP('%Y%m', user.creation_date) AS int64) >= 201601
  AND CAST(FORMAT_TIMESTAMP('%Y%m', user.creation_date) AS int64) <= 201606

"""
query_job = client.query(query)
results = query_job.result()
lcv_id = results.to_dataframe()

lcv_id.dropna(inplace=True)
lcv_id['id'] = lcv_accepted['id'].astype(int)
lcv_id.sort_values('id',inplace=True)

with open('./data/processed/lcv_id.pkl', 'wb') as picklefile:
    pickle.dump(lcv_id, picklefile)

In [30]:
lcv_id.sort_values('id',inplace=True)

(764974, 1)

In [36]:
lcv_answer_all = pd.merge(lcv_id, lcv_answer, how='left', left_on='id', right_on='id' )

In [41]:
lcv_question_all = pd.merge(lcv_id, lcv_question, how='left', left_on='id', right_on='id' )